## Covid-19 Data Exploration and Cleanup

In [1]:
# Import libraries
import matplotlib.pyplot as plt
import pandas as pd
import gmaps
import json

# Import API key
from api_keys import g_key

# Output File (CSV), need 3 files for 3 sets of data for comparison
output_data_file_1 = "output_data/covid_df_1.csv"
output_data_file_2 = "output_data/covid_df_2.csv"
output_data_file_3 = "output_data/covid_df_3.csv"

In [2]:
#File 1 reference

# Load file 1 to read
covid_data_1 = "Resources/daily.csv"

# Read csv file into DataFrame, set state as index
covid_df_1 = pd.read_csv(covid_data_1, index_col = "state")
#covid_df_1 = pd.read_csv(covid_data_1)
covid_df_1

,date,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
state,,,,,,,,,,,,,,,,,,,,,
AK,20200717,2081.0,163333.0,NaN,32.0,NaN,NaN,NaN,0.0,NaN,...,165414,0,0,e904def0e1f67ac528572f0506eef868c163a3f3,0,0,0,0,0,NaN
AL,20200717,63091.0,500024.0,NaN,1416.0,7584.0,NaN,967.0,NaN,521.0,...,563115,35,0,5bb063f5f4fe7c02d748b1fee5b811ac19d8f8ba,0,0,0,0,0,NaN
AR,20200717,31762.0,375735.0,NaN,464.0,2070.0,NaN,NaN,97.0,298.0,...,407497,12,76,c1ffb0bb728cd1ac03f834ec70d20070b443f395,0,0,0,0,0,NaN
AS,20200717,0.0,1037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1037,0,0,b0c23fca3c4bb610a1625c60aca491eab6a71161,0,0,0,0,0,NaN
AZ,20200717,138523.0,626445.0,NaN,3466.0,6402.0,944.0,NaN,687.0,NaN,...,764968,91,106,fc227a78fd73620678a4d4cb836c128b38541778,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WA,20200126,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,0,0,7acb526e14f20a29cc74a0b32a37328bc6eac6c2,0,0,0,0,0,NaN
WA,20200125,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,0,0,9b52ca94dd2a996822542ea5f17a7363e7ad91cf,0,0,0,0,0,NaN
WA,20200124,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,0,0,094154f68e74bfc30b977cdee888f9c07be4360e,0,0,0,0,0,NaN


In [3]:
# Narrow down columns of covid_df_1 to state, date, positive
df_1 = covid_df_1[["date", "positive"]]
df_1

,date,positive
state,,
AK,20200717,2081.0
AL,20200717,63091.0
AR,20200717,31762.0
AS,20200717,0.0
AZ,20200717,138523.0
...,...,...
WA,20200126,2.0
WA,20200125,2.0
WA,20200124,2.0


In [4]:
# Rename columns for df_1 positive to cases to be consistent with other dataframes
df_1.rename(columns = {"positive": "cases"})

,date,cases
state,,
AK,20200717,2081.0
AL,20200717,63091.0
AR,20200717,31762.0
AS,20200717,0.0
AZ,20200717,138523.0
...,...,...
WA,20200126,2.0
WA,20200125,2.0
WA,20200124,2.0


In [5]:
# Filter out GA, TX and WA
states = ["GA", "TX", "WA"]
filter_df_1  = df_1[df_1.index.isin(states)]
filter_df_1

,date,positive
state,,
GA,20200717,135183.0
TX,20200717,307572.0
WA,20200717,44313.0
GA,20200716,131275.0
TX,20200716,292656.0
...,...,...
WA,20200126,2.0
WA,20200125,2.0
WA,20200124,2.0


In [6]:
# Rename state codes to Georgia, Texas and Washington
filter_df_1.rename(index={"GA": "Georgia", "TX": "Texas", "WA":"Washington"})

,date,positive
state,,
Georgia,20200717,135183.0
Texas,20200717,307572.0
Washington,20200717,44313.0
Georgia,20200716,131275.0
Texas,20200716,292656.0
...,...,...
Washington,20200126,2.0
Washington,20200125,2.0
Washington,20200124,2.0


In [14]:
# Save to csv file
filter_df_1.to_csv("output_data/covid_df_1.csv")

In [12]:
# For reference for next section
#Group by state, date and positive, use this when plotting in next section, can't save groupby as csv 
group_df_1 = filter_df_1.groupby(["state", "date"])
group_df_1.sum()

positive
state date              
GA    20200304       2.0
      20200305       2.0
      20200306       2.0
      20200307       6.0
      20200308       7.0
...                  ...
WA    20200713   40656.0
      20200714   41757.0
      20200715   42304.0
      20200716   43046.0
      20200717   44313.0

[450 rows x 1 columns]

In [15]:
# Load file 2 to read
covid_data_2 = "Resources/us-counties.csv"

# Read csv file into DataFrame, set state as index
covid_df_2 = pd.read_csv(covid_data_2, index_col = "state")
covid_df_2

,date,county,fips,cases,deaths
state,,,,,
Washington,1/21/2020,Snohomish,53061.0,1,0
Washington,1/22/2020,Snohomish,53061.0,1,0
Washington,1/23/2020,Snohomish,53061.0,1,0
Illinois,1/24/2020,Cook,17031.0,1,0
Washington,1/24/2020,Snohomish,53061.0,1,0
...,...,...,...,...,...
Wyoming,7/19/2020,Sweetwater,56037.0,173,2
Wyoming,7/19/2020,Teton,56039.0,208,1
Wyoming,7/19/2020,Uinta,56041.0,221,0


In [16]:
# Narrow down columns of covid_df_2 to state, date, cases
df_2 = covid_df_2[["date", "cases"]]
df_2

,date,cases
state,,
Washington,1/21/2020,1
Washington,1/22/2020,1
Washington,1/23/2020,1
Illinois,1/24/2020,1
Washington,1/24/2020,1
...,...,...
Wyoming,7/19/2020,173
Wyoming,7/19/2020,208
Wyoming,7/19/2020,221


In [18]:
# Filter out Georgia, Texas and Washington
states = ["Georgia", "Texas", "Washington"]
filter_df_2  = df_2[df_2.index.isin(states)]
filter_df_2

,date,cases
state,,
Washington,1/21/2020,1
Washington,1/22/2020,1
Washington,1/23/2020,1
Washington,1/24/2020,1
Washington,1/25/2020,1
...,...,...
Washington,7/19/2020,5
Washington,7/19/2020,294
Washington,7/19/2020,780


In [19]:
# Save to csv file
filter_df_2.to_csv("output_data/covid_df_2.csv")

In [20]:
# For reference for next section
#Group by state, date and positive, use this when plotting in next section, can't save groupby as csv 
group_df_2 = filter_df_2.groupby(["state", "date"])
group_df_2.sum()

cases
state      date            
Georgia    3/10/2020     22
           3/11/2020     31
           3/12/2020     31
           3/13/2020     41
           3/14/2020     65
...                     ...
Washington 7/5/2020   37468
           7/6/2020   38517
           7/7/2020   39063
           7/8/2020   39661
           7/9/2020   40369

[480 rows x 1 columns]

In [21]:
# Load file 3 to read
covid_url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"

# Read csv file into DataFrame, set state as index
covid_df_3 = pd.read_csv(covid_url, index_col = "state")
covid_df_3

,date,fips,cases,deaths
state,,,,
Washington,2020-01-21,53,1,0
Washington,2020-01-22,53,1,0
Washington,2020-01-23,53,1,0
Illinois,2020-01-24,17,1,0
Washington,2020-01-24,53,1,0
...,...,...,...,...
Virginia,2020-07-20,51,78375,2031
Washington,2020-07-20,53,49949,1521
West Virginia,2020-07-20,54,5142,100


In [22]:
# Narrow down columns of covid_df_2 to state, date, cases
df_3 = covid_df_3[["date", "cases"]]
df_3

,date,cases
state,,
Washington,2020-01-21,1
Washington,2020-01-22,1
Washington,2020-01-23,1
Illinois,2020-01-24,1
Washington,2020-01-24,1
...,...,...
Virginia,2020-07-20,78375
Washington,2020-07-20,49949
West Virginia,2020-07-20,5142


In [23]:
# Filter out Georgia, Texas and Washington
states = ["Georgia", "Texas", "Washington"]
filter_df_3  = df_3[df_3.index.isin(states)]
filter_df_3

,date,cases
state,,
Washington,2020-01-21,1
Washington,2020-01-22,1
Washington,2020-01-23,1
Washington,2020-01-24,1
Washington,2020-01-25,1
...,...,...
Texas,2020-07-19,338036
Washington,2020-07-19,48601
Georgia,2020-07-20,132788


In [24]:
# Save to csv file
filter_df_3.to_csv("output_data/covid_df_3.csv")

In [25]:
# For reference for next section
#Group by state, date and positive, use this when plotting in next section, can't save groupby as csv 
group_df_3 = filter_df_3.groupby(["state", "date"])
group_df_3.sum()

cases
state      date             
Georgia    2020-03-02      2
           2020-03-03      2
           2020-03-04      2
           2020-03-05      2
           2020-03-06      3
...                      ...
Washington 2020-07-16  46268
           2020-07-17  47226
           2020-07-18  47889
           2020-07-19  48601
           2020-07-20  49949

[483 rows x 1 columns]

In [ ]:
# Pull population data
# Downloaded data from https://www.kaggle.com/headsortails/covid19-us-county-jhu-data-demographics/data?select=us_county.csv
# Saved as csv named population datasets

#Population data
#Load file to read
pop_file = ("Resources/population datasets.csv")

# Read csv file into DataFrame
pop_df = pd.read_csv(pop_file)
pop_df

In [ ]:
# Build dataset with state data
# Reference https://www.washingtonpost.com/graphics/2020/national/states-reopening-coronavirus-map/

data = {'State':['Georgia', 'Texas', 'Washington'], 
        'Governor':['Brian Kemp (R)', 'Greg Abbott (R)', 'Jay Inslee (D)'],
       'Statewide Shelter in Place Date':['4/3/20', '4/2/20', '3/23/20'],
       'Phase One Reopening Date':['4/24/20', '5/1/20', '5/4/20'],
       'Total cases (2/29/20 to 7/21/20)': [145575, 332434, 47743],
       'Population': [10297484, 27885195, 7294336]} 
  
# Create DataFrame 
state_df = pd.DataFrame(data) 
  
# Print the output. 
state_df